# PROGETTO DATAMAN: script relativo allo scraping della cronaca del match PSG-Bayern



*   Importazione dei pacchetti



In [1]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import json
import re

* Varaibilizzazione URL dal sito di Repubblica: [PSG-Bayern](https://www.repubblica.it/sport/live/calcio/europa/champions-league/2019/diretta/psg-bayern%20m.)
* Estrazione dati dalla pagina con BeautifulSoup

In [2]:
url='https://www.repubblica.it/sport/live/calcio/europa/champions-league/2019/diretta/psg-bayern%20m.'
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

* Estrazione della sezione d'interesse dal sito, in formato json

In [3]:
data = soup.find("script",{"type":"application/ld+json"})
oJson = json.loads(data.text)

* Pulizia dell'estratto e disposizione degli eventi in una lista relativamente al minutaggio

In [4]:
com=re.compile(r'\xa0*')
text=com.sub('',oJson['articleBody'])

In [5]:
text_1=re.findall(r'>(.+?)<',text)

com1=re.compile(r'<br>|<b>')
com3=re.compile(r"<div>")
ls=[]
for i in range(0,len(text_1)):
    if com3.search(text_1[i]):
        del text_1[i:len(text_1)]
        break
    ls.append(com1.sub('',text_1[i]))

In [6]:
com2=re.compile("^(?![0-9]+\'\:)")
for k in range(0,3):
    i=0
    while True:
        if com2.search(ls[i]):
            x=ls.index(ls[i])-1
            xx=ls.index(ls[i])+1
            ls[x:xx]=[''.join(ls[x:xx])]
        i+=1
        if (i==(len(ls))) | (i==len(ls)-1):
            break

In [7]:
ls=[i for i in ls if re.search(r':', i) is not None]

In [8]:
if re.search("(0':)|calcio d'inizio",ls[len(ls)-1]) is None:
    ls.append("0': Calcio d'inizio")

* Creazione delle variabili minuto e testo utili alla creazione del dataframe

In [9]:
minuto=[]
testo=[]
for i in ls:
    x=i.split(':')
    minuto.append(x[0])
    testo.append(x[1])

In [10]:
last_min=re.search("^9[0-9]\'",text)
if re.search("^9[0-9]\'",text) is not None:
    minuto.insert(0,last_min.group(0))
    testo.insert(0,'Fine partita')

In [11]:
minuto.reverse()
testo.reverse()

In [12]:
dataframe=pd.DataFrame({'minuto':minuto,'testo':testo})

* Predisposizione di espressioni regolari utili all'identificazione nel testo di parole chiave riguardanti il relativo commento della cronaca allo scopo di creare le variabili:

> * **squadre**: quelle coinvolte nel commento
> * **soggetti**: i giocatori menzionati nel commento
> * **eventi**: termini che identificano l'evento accaduto nel rispettivo minuto di gioco


In [13]:
com_psg=re.compile('navas|kehrer|bernat|marquinhos|silva|thiago silva|kimpembe|herrera|paredes|neymar|di mar[ií]a|mbappé|verratti|kurzawa|draxler|choupo-moting|psg')
com_bayern=re.compile('neuer|kimmich|davies|alc[aá]ntara|boateng|alaba|gnabry|goretzka|lewandowski|muller|com[ea]n|perisic|tolisso|coutinho|sule|bayern')
com_totale=re.compile('navas|kehrer|bernat|marquinhos|silva|thiago silva|thiago|kimpembe|herrera|paredes|neymar|di mar[ií]a|mbappé|verratti|kurzawa|draxler|choupo-moting|neuer|kimmich|davies|alc[aá]ntara|boateng|alaba|gnabry|goretzka|lewandowski|muller|coman|perisic|tolisso|coutinho|sule')

In [14]:
com_inizio=re.compile("riparte|calcio d'inizio|ripresa")
com_palo=re.compile("palo")
com_pallonetto=re.compile('pallonetto')
com_chiusura=re.compile('chiuso|chiusura|chiude')
com_var=re.compile('var ')
com_contrasto=re.compile('contrasto')
com_deviazione=re.compile('deviazione|deviato')
com_tiro=re.compile('ci prova|calcia|destro|fuori|girata|calcia|conclusione|sassata|spara|spreca|sinistro|manda il pallone|manda la palla')
com_fuorigioco=re.compile('fuorigioco|offside') 
com_giallo=re.compile('giallo|ammonito|ammonizione')
com_traversone=re.compile('traversone')
com_mano=re.compile('braccio|mano')
com_scivolata=re.compile('scivolata')
com_rigore=re.compile('rigore per')
com_area_rigore=re.compile('area di rigore')
com_muro=re.compile('murato|barriera')
com_fuori=re.compile('fuori|sul fondo')
com_gol=re.compile('([0-9]+)\-([0-9]+)|il gol') 
com_assist=re.compile('assist |serve|lancio')
com_gol_null=re.compile('gol annullato')
com_pressing=re.compile('pressing|pressa')
com_testa=re.compile('testa')
com_cooling=re.compile('cooling break')
com_rischio=re.compile('rischia|spinta') 
com_cross=re.compile('cross|crossa')
com_corner=re.compile("calcio d'angolo|corner|in angolo")
com_fermo=re.compile('gioco fermo|problemi per|rissa|gioco era fermo')
com_sostituzioni=re.compile('sostituzione|sostituzioni|cambio|deve uscire')
com_punizione=re.compile('punizione|fallo')
com_fuori=re.compile('fuori|fallo laterale|out|sul fondo')
com_recupero=re.compile('recupero|per altri ([0-9]+)')
com_primo_tempo=re.compile('fine primo tempo|finisce il primo tempo')
com_parata=re.compile('tra le braccia|parata|pugni|blocca |presa bassa|pugno|presa alta|respinge|respingere|respinto|respinta|in uscita')
com_fine=re.compile('fine partita')

dizionario={"calcio d'inizio":com_inizio,"pressing":com_pressing,"cooling break":com_cooling,"traversone":com_traversone,"fuori":com_fuori,"var":com_var,'chiusura':com_chiusura,'pallonetto':com_pallonetto,'deviazione':com_deviazione,
            'palo':com_palo,'tiro':com_tiro,'fuorigioco':com_fuorigioco,"scivolata":com_scivolata,'contrasto':com_contrasto,"fine primo tempo":com_primo_tempo,
            'cartellino giallo':com_giallo,'colpo di testa':com_testa,'rigore':com_rigore,'area di rigore':com_area_rigore,'gol':com_gol,'assist':com_assist,
           'gol annullato':com_gol_null,'muro':com_muro,'gioco pericoloso':com_rischio,'cross':com_cross,'corner':com_corner,'parata':com_parata,
           'gioco fermo':com_fermo,'sostituzione':com_sostituzioni,'punizione':com_punizione,'recupero':com_recupero,'fine partita':com_fine,'mano':com_mano}

In [15]:
def findall(compiled, string):
    lista=[]
    while True:
        match = compiled.search(string)
        if not match:
            break
        lista.append(match.group(0))
        string = string[match.end():]
    return lista

In [16]:
dataframe['squadre']=0
dataframe['soggetti']=0
dataframe['eventi']=0

for i in range(0,len(dataframe['testo'])):
    frase=dataframe.testo[i].lower()
    x=com_psg.search(frase)
    y=com_bayern.search(frase)
    z=findall(com_totale,frase)
    status_x=False
    status_y=False
    
    eventi=[]
    for j,k in dizionario.items(): 
        if k.search(frase):
            eventi.append(j)
            if j=="calcio d'inizio":
                if i==0:
                    eventi=[]
                    eventi.append("inizio primo tempo")
                elif (i>0) & (dataframe.eventi[i-1]=='cooling break'):
                    eventi=[]
                    eventi.append("ripresa dal break")
                else:
                    eventi.append("calcio d'inizio")
                
                
    if ('rigore' in eventi) & ('area di rigore' in eventi):
        eventi.remove('rigore')
        eventi.remove('area di rigore')
    
    if ('var' in eventi) & ('gol' in eventi):
        eventi.remove('gol')
    
    if ('rigore' in eventi) & ('punizione' in eventi):
        eventi.remove('punizione')
    
    if ('fallo' in eventi) & ('fuori' in eventi):
        eventi.remove('fallo')
    
    if ('gol' in eventi) & (re.search('sarebbe',frase) is not None):
        eventi=[]
    
    if (re.search('entra',frase) is not None) & (re.search('esce',frase) is not None):
        eventi=[]
        eventi.append('sostituzione')
    
    if re.search('chiuso in angolo|anticipato in angolo',frase) is not None:
        eventi.remove('corner')
        
    if x:
        dataframe.squadre[i]='psg'
        status_x=True
    if y:
        dataframe.squadre[i]='bayern'
        status_y=True
    if status_x & status_y:
        dataframe.squadre[i]='psg, bayern'
    
    if z:
        dataframe.soggetti[i]=str(', '.join(list(set(z))))
        
    if "fine primo tempo" in eventi:
        eventi=[]
        eventi.append('fine primo tempo')
        dataframe.soggetti[i]=str('')
    
    
    dataframe.eventi[i]=str(', '.join(eventi))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55:

* Perfezionamento del dataset a fini di leggibilità e creazione di una variabile **minuto effettivo** utile a scopi futuri

In [17]:
for i in dataframe.minuto:
    if re.search('45',i) is not None:
        index0=dataframe[dataframe.minuto==i].index[0]+1

In [18]:
index1=dataframe[dataframe.eventi=='fine primo tempo'].index[0]+1
for i in range(index0,index1):
    x=int(dataframe.minuto[i].replace("'",""))-45
    dataframe.minuto[i]="45' + "+str(x)+"'"

In [19]:
index2=dataframe[dataframe.eventi=='recupero'].index
index2=index2[index2>70][0]+1
index3=len(dataframe)
for i in range(index2,index3):
    x=int(dataframe.minuto[i].replace("'",""))-90
    dataframe.minuto[i]="90' + "+str(x)+"'"

In [20]:
compiler=re.compile('^(?P<minuto>\d+\')[\s\+\s]*(?P<recupero>\d\')*$')
lista_1=[]
for i in dataframe.minuto:
    recupero1=0
    recupero2=0
    searcher=compiler.search(i)
    min_partita=re.search('\d+',searcher.group('minuto')).group(0)
    if searcher.group('recupero') != None:
        if dataframe[dataframe.minuto==i].index[0]>50:
            recupero2=re.search('\d',searcher.group('recupero')).group(0)
        else:
            recupero1=re.search('\d',searcher.group('recupero')).group(0)
    lista_1.append(int(min_partita)+int(recupero1)+int(recupero2))

In [21]:
dataframe.insert(loc=1,column='minuto_effettivo',value=lista_1)

* Creazione dei un file csv a partire dal dataframe ultimato

In [22]:
dataframe.to_csv('FinaleCronaca.csv')